In [1]:
import os
from dotenv import load_dotenv
load_dotenv


<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [2]:
from langchain_groq import ChatGroq
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="llama-3.3-70b-versatile")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026ABA118400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026ABA0FBDC0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import (
    AIMessage, HumanMessage, SystemMessage
)

In [5]:
speech="""
People across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.

In our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get their houses. And he should not need to make rounds of the government offices for this purpose. The government should reach him. Since you have assigned this responsibility to me, about four crore families have got their ‘pucca’ houses. However, I have encountered cases where someone is left out of the government benefits. Therefore, I have decided to tour the country again, to listen to people’s experiences with government schemes, to understand whether they received the intended benefits, and to ensure that the programs are reaching everyone as planned without paying any bribes. We will get the real picture if we visit them again. Therefore, this ‘Viksit Bharat Sankalp Yatra’ is, in a way, my own examination. I want to hear from you and the people across the country whether what I envisioned and the work I have been doing aligns with reality and whether it has reached those for whom it was meant.

It is crucial to check whether the work that was supposed to happen has indeed taken place. I recently met some individuals who utilized the Ayushman card to get treatment for serious illnesses. One person met with a severe accident, and after using the card, he could afford the necessary operation, and now he is recovering well. When I asked him, he said: “How could I afford this treatment? Now that there is the Ayushman card, I mustered courage and underwent an operation. Now I am perfectly fine.”  Such stories are blessings to me.

The bureaucrats, who prepare good schemes, expedite the paperwork and even allocate funds, also feel satisfied that 50 or 100 people who were supposed to get the funds have got it. The funds meant for a thousand villages have been released. But their job satisfaction peaks when they hear that their work has directly impacted someone’s life positively. When they see the tangible results of their efforts, their enthusiasm multiplies. They feel satisfied. Therefore, ‘Viksit Bharat Sankalp Yatra’ has had a positive impact on government officers. It has made them more enthusiastic about their work, especially when they witness the tangible benefits reaching the people. Officers now feel satisfied with their work, saying, “I made a good plan, I created a file, and the intended beneficiaries received the benefits.” When they find that the money has reached a poor widow under the Jeevan Jyoti scheme and it was a great help to her during her crisis, they realise that they have done a good job. When a government officer listens to such stories, he feels very satisfied.

There are very few who understand the power and impact of the ‘Viksit Bharat Sankalp Yatra’. When I hear people connected to bureaucratic circles talking about it, expressing their satisfaction, it resonates with me. I’ve heard stories where someone suddenly received 2 lakh rupees after the death of her husband, and a sister mentioned how the arrival of gas in her home transformed her lives. The most significant aspect is when someone says that the line between rich and poor has vanished. While the slogan ‘Garibi Hatao’ (Remove Poverty) is one thing, but the real change happens when a person says, “As soon as the gas stove came to my house, the distinction between poverty and affluence disappeared.
"""

In [6]:
chat_message=[
    SystemMessage(content="You are expert with experise in summarizing speeched"),
    HumanMessage(content=f"Please provide a short and concisse summary of the follow speech:\n Text:{speech}")
]

In [7]:
llm.get_num_tokens(speech)

909

In [9]:
llm.invoke(chat_message).content

"Here is a concise summary of the speech:\n\nThe speaker, a Member of Parliament, is participating in the 'Viksit Bharat Sankalp Yatra' (Developed India Resolution Journey) to ensure that government schemes reach their intended beneficiaries. They aim to listen to people's experiences, identify gaps, and assess the impact of programs like the Prime Minister's housing scheme and Ayushman card. The speaker emphasizes the importance of tangible results and the satisfaction of government officers when their work positively affects people's lives. They share stories of individuals who have benefited from these schemes, highlighting the transformative power of government initiatives in reducing poverty and bridging the gap between the rich and the poor. The 'Viksit Bharat Sankalp Yatra' is seen as a means to evaluate the effectiveness of these schemes and to motivate government officers to work towards a more developed India."

the above one is the first technique to summarize the text where i give the chat_message.
Now we will try another technique for long documents.


## Prompt Template Text Summarization


In [29]:
from langchain_classic.chains import LLMChain
from langchain_core.prompts import PromptTemplate
generictemplate = """
Write a summary of the following speech:
Speech:{speech}
Translate the precise summary to {language}
"""
prompt = PromptTemplate(
    input_variables=["speech", "language"],
    template=(
        "You are a professional multilingual summarizer. "
        "Summarize the following speech in {language} only. "
        "Do not translate back to English. Write naturally in {language}.\n\n"
        "Speech:\n{speech}\n\n"
        "Summary ({language}):"
    )
)


In [30]:
complete_prompt=prompt.format(speech=speech, language="French")
complete_prompt

'You are a professional multilingual summarizer. Summarize the following speech in French only. Do not translate back to English. Write naturally in French.\n\nSpeech:\n\nPeople across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.\n\nIn our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri A

In [31]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary = llm_chain.invoke({'speech':speech, 'language':'hindi'})
summary

C:\Users\Vaishali\AppData\Local\Temp\ipykernel_22516\1838060946.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(llm=llm,prompt=prompt)


{'speech': '\nPeople across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra’ (Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.\n\nIn our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made. However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government’s plans reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister’s housing scheme), then those who are living in jhuggis and slums should get their houses. And he should not need to make rounds 

## StuffDocumentChain Text Summarization


In [23]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
docs


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [36]:
template = """
Write a concise and short summary of the following speech,
Speech:{text}
"""

prompt = PromptTemplate(input_variables=["text"],template=template)

In [37]:
from langchain_classic.chains import load_summarize_chain 

# Tell load_summarize_chain the variable name the chain should look for
chain = load_summarize_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
    verbose=True
)
output_summary = chain.invoke(docs)
output_summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise and short summary of the following speech,
Speech:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online int

{'input_documents': [Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who 

### MapReduce to summarize large documents

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
final_documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [39]:
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [40]:
final_prompt='''
Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire speech with these important points.\nAdd a Motivation Title,Start the precise summary with an introduction and provide the summary in number \npoints for the speech.\nSpeech:{text}\n\n')

In [41]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output=summary_chain.run(final_documents)
output

C:\Users\Vaishali\AppData\Local\Temp\ipykernel_22516\2349673758.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  output=summary_chain.run(final_documents)




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:`A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have m

Token indices sequence length is longer than the specified maximum sequence length for this model (1976 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:The speech is A.P.J. Abdul Kalam's departing speech as the President of India, after completing his five-year tenure. He expresses his gratitude to the people of India and shares his experiences, highlighting 10 important messages that he learned during his time in office. These messages include:

1. Accelerating development to meet the aspirations of the youth
2. Empowering villages
3. Utilizing rural core competence for competitiveness
4. Focusing on agricultural growth
5. Overcoming problems through determination and partnership
6. Showing courage in the face of calamities
7. Promoting connectivity for societal transformation
8. Defending the nation with pride
9. Fostering a youth movement for a developed India by 2

'**Motivation Title: "Empowering a Nation: A Call to Action"**\n\nIn his departing speech as the President of India, A.P.J. Abdul Kalam reflects on his experiences and shares valuable lessons learned during his tenure. The speech is a heartfelt expression of gratitude to the people of India and a call to action for collective effort towards a brighter future. Here is a summary of the key points:\n\n1. **Accelerate Development**: The need to accelerate development to meet the aspirations of the youth, with a focus on empowering villages and utilizing rural core competence for competitiveness.\n2. **Empowering Villages**: The importance of empowering village councils and providing physical connectivity to transform the 600,000 villages of India and make the country more competitive.\n3. **Agricultural Growth**: The need to double agricultural production with reduced resources and improve the economic conditions of the nation through the principle of "Seed to Food".\n4. **Overcoming Probl

### Refine Chain for Summarization


In [43]:
chain = load_summarize_chain(
    llm = llm,
    chain_type = "refine",
    verbose = True
)
output_summary = chain.run(final_documents)
output_summary



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

"The original summary is mostly accurate, but it can be refined slightly with the new context provided. Here's a refined summary:\n\nIn his departing speech, A.P.J. Abdul Kalam reflects on his 5-year tenure as President, expressing gratitude for the associations he made and the cooperation and support he received from citizens. He reiterates his vision for India, including accelerating development, empowering villages, and overcoming problems through partnership, with a focus on the youth's aspiration for a developed India by 2020. Additionally, he emphasizes the importance of preserving the country's cultural heritage, promoting equitable distribution of resources, and fostering a sense of self-confidence and unity among the people, aiming to connect the hearts and minds of India's multicultural society. He concludes by reaffirming his commitment to the mission of making India a developed nation by 2020."